In [137]:
import argparse
from train import *
import datetime

config = argparse.Namespace()
config.device = 'cuda'
config.L = 15
config.d = 512
config.maxlen = 15
config.batch_size = 1024
config.data_dir = 'Food-Kitchen'
config.tau = 9
config.tau2 = 7
config.lamb = 0.6
config.neg_samples = 99
config.model = 'LLM_ECDSR'
config.use_llm_emb = True
config.use_llm_user_emb = True
config.use_context_conv = True

test_data_x = DataSet(config.data_dir, config.batch_size, config, evaluation=1, pred_domain='x')
test_data_y = DataSet(config.data_dir, config.batch_size, config, evaluation=1, pred_domain='y')

test_loader_x = DataLoader(test_data_x, config.batch_size)
test_loader_y = DataLoader(test_data_y, config.batch_size)

config.item_num = config.source_item_num + config.target_item_num + 1

print("Data loading done!")

# 模型初始化
model = LeCDSR(config)

model.load_state_dict(torch.load(f'saved_models/LLMECDSR_test_{config.data_dir}.pth'))
model.to(device)

print("recommendation: -------------------------------------------------------------------")
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print(config)
print(device)

HR, MRR, NDCG = evaluation(model, test_loader_x, test_loader_y, topk=10)
print("---- \nX domain valid data evaluation: \n"
        "MRR    , NDCG@5 , NDCG@10, HR@1   , HR@5   , HR@10 ")
print("{:.5f}, {:.5f}, {:.5f}, {:.5f}, {:.5f}, {:.5f}"
        .format(MRR[0][0], NDCG[0][1], NDCG[0][2], HR[0][0], HR[0][1], HR[0][2]))
print("---- \nY domain valid data evaluation: \n"
        "MRR    , NDCG@5 , NDCG@10, HR@1   , HR@5   , HR@10 ")
print("{:.5f}, {:.5f}, {:.5f}, {:.5f}, {:.5f}, {:.5f}"
        .format(MRR[1][0], NDCG[1][1], NDCG[1][2], HR[1][0], HR[1][1], HR[1][2]))
print("---- ")

Data loading done!
recommendation: -------------------------------------------------------------------
2025-05-27 10:58:26
Namespace(device='cuda', L=15, d=512, maxlen=15, batch_size=1024, data_dir='Food-Kitchen', tau=9, tau2=7, lamb=0.6, neg_samples=99, model='LLM_ECDSR', use_llm_emb=True, use_llm_user_emb=True, use_context_conv=True, source_item_num=29207, target_item_num=34886, user_num=16579, item_num=64094)
cuda
---- 
X domain valid data evaluation: 
MRR    , NDCG@5 , NDCG@10, HR@1   , HR@5   , HR@10 
0.14448, 0.15617, 0.17157, 0.09641, 0.21078, 0.25803
---- 
Y domain valid data evaluation: 
MRR    , NDCG@5 , NDCG@10, HR@1   , HR@5   , HR@10 
0.08485, 0.09117, 0.10585, 0.05209, 0.12914, 0.17417
---- 


In [138]:
dataloader_iter = iter(test_loader_x)
first_sample = next(dataloader_iter)

inputs = [torch.vstack(b).T.to(device) for b in first_sample]
seq = inputs[0]         # cross-domain 序列
ground = inputs[1]      # ground truth
position = inputs[2]    # cross-domain 位置序列
user = inputs[3]        # user
neg = inputs[4]        # negative samples

items_to_predict = torch.cat((ground, neg), 1)      # candidate items

# candidate items' scores
prediction_score = model(seq, position, user, items_to_predict)

prediction_score = prediction_score.cpu().data.numpy().copy()
# 根据评分排序
sorted_indices = np.argsort(prediction_score)
top_ten_indices = sorted_indices[:, -10:]
items_to_predict = items_to_predict.cpu().data.numpy().copy()
ground = ground.cpu().data.numpy().copy()

In [139]:
rec_result = []
for i in range(920):
    rec_result.append(items_to_predict[i][top_ten_indices[i]])
user = user.cpu().data.numpy().copy()

In [99]:
import pandas as pd
import gzip


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

movie_path = 'dataset/generate_LLM_embeddings/Movie-Book/meta_Movies_and_TV.json.gz'
book_path = 'dataset/generate_LLM_embeddings/Movie-Book/meta_Books.json.gz'
food_path = "dataset/generate_LLM_embeddings/Food-Kitchen/meta_Grocery_and_Gourmet_Food.json.gz"
kitchen_path = 'dataset/generate_LLM_embeddings/Food-Kitchen/meta_Home_and_Kitchen.json.gz'

def get_item_list(file_path):
    """获取商品列表，商品id已设为id，使用iloc查询指定商品"""
    # 逐行读取文件
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # 处理每一行
    data = []
    for line in lines:
        row = line.strip().split('\t')  # 按制表符分割
        data.append(row)
    # 转换为DataFrame
    column = ['origin_index', 'asin', 'id']
    df = pd.DataFrame(data, columns=column)
    df.set_index('id', inplace=True)
    return df

def get_user_list(file_path):
    """使用dataset/Food-Kitchen/userlist.txt, 按行读取每一个用户id
    """
    # 逐行读取文件
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # 处理每一行
    data = []
    for line in lines:
        row = line.strip().split('\t')  # 按制表符分割
        data.append(row)
    # 转换为DataFrame
    column = ['asin', 'id']
    df = pd.DataFrame(data, columns=column)
    return df


file_path_A = 'dataset/Food-Kitchen/Alist.txt'
file_path_B = 'dataset/Food-Kitchen/Blist.txt'
df_item_Alist = get_item_list(file_path_A)
df_item_Blist = get_item_list(file_path_B)
# item数量直接加和为两域之和
df_item = pd.concat([df_item_Alist, df_item_Blist])
df_meta_data_A = getDF(food_path)
df_meta_data_B = getDF(kitchen_path)
df_meta_data = pd.concat([df_meta_data_A, df_meta_data_B])
df_item_meta_data = pd.merge(df_item, df_meta_data, on='asin', how='left')

In [119]:
df_item_meta_data

,origin_index,asin,description,title,imUrl,related,salesRank,categories,price,brand
0,6407,B0028A8ZEI,Distinctively unique and different ... And unb...,The Peanut Shop of Williamsburg Handcooked Lig...,http://ecx.images-amazon.com/images/I/51dTwdLv...,"{'also_bought': ['B002S5HJ1I', 'B00IH1NDH6', '...",{'Grocery & Gourmet Food': 50292},[[Grocery & Gourmet Food]],19.99,The Peanut Shop of Williamsburg
1,15157,B001D9OVQ6,"Queen Anne Cordial Cherries Milk Chocolate, 10...","Queen Anne Cordial Cherries Milk Chocolate, 10...",http://ecx.images-amazon.com/images/I/41PdUk84...,"{'also_bought': ['B006JVU1X0', 'B004N5J5F4', '...",{'Grocery & Gourmet Food': 77480},[[Grocery & Gourmet Food]],15.00,Queen Ann
2,9314,B001QY75DE,Snack on these antioxidant packed wonders of c...,Emily's Roasted &amp; Salted Pecans 30oz,http://ecx.images-amazon.com/images/I/41eCbvPj...,"{'also_bought': ['B000X3H46C', 'B0060ZISVM', '...",{'Grocery & Gourmet Food': 84356},[[Grocery & Gourmet Food]],NaN,NaN
3,22989,B009SB77PI,NaN,NEW Queen Anne Milk Chocolate French Vanilla C...,http://ecx.images-amazon.com/images/I/5173HWqy...,"{'also_viewed': ['B001D9OVQ6', 'B006JVU1X0', '...",{'Grocery & Gourmet Food': 162084},[[Grocery & Gourmet Food]],16.95,NaN
4,6411,B004P30SWI,A pack of 10 Nespresso Coffee Machine Capsules...,PACK OF 10 NESPRESSO RISTRETTO COFFEE CAPSULES...,http://ecx.images-amazon.com/images/I/31aSIOgx...,"{'also_bought': ['B004P7RGMY', 'B004OXADZQ', '...",{'Grocery & Gourmet Food': 653},[[Grocery & Gourmet Food]],11.76,Nespresso
...,...,...,...,...,...,...,...,...,...,...
64088,89002,B00198BG78,"It's the New Fondue! Gather the children, inv...",Wimmeley PD601 PizzaDome Portable Italian Bric...,http://ecx.images-amazon.com/images/I/41EGOdZr...,"{'also_viewed': ['B00005IBXJ', 'B0046R1B6A', '...",{'Kitchen & Dining': 136472},"[[Home & Kitchen, Kitchen & Dining, Bakeware, ...",NaN,NaN
64089,83544,B003VWBRD8,"A sand timer, also known as an hourglass, is a...",G.W. Schleidt 40015CH-P Sand Timer 15 Minute P...,http://ecx.images-amazon.com/images/I/41P3odX-...,"{'also_bought': ['B003LM4JJM', 'B003Y5BIMC', '...","{'Patio, Lawn & Garden': 16982}","[[Home & Kitchen, Kitchen & Dining, Kitchen Ut...",15.26,G.W. Schleidt
64090,67859,B0019Q8AGK,Cross Over Powermate: A hybrid power nozzle co...,"Kenmore Canister Vacuum, Intuition",http://ecx.images-amazon.com/images/I/21ovBw6p...,"{'also_viewed': ['B00642T5LI', 'B005OSDE8I', '...",{'Home &amp; Kitchen': 701935},"[[Home & Kitchen, Vacuums & Floor Care, Vacuum...",NaN,NaN
64091,98620,B004BHTCE8,The Stanton collection will give your contempo...,Coaster Stanton Contemporary Dining Table in B...,http://ecx.images-amazon.com/images/I/41T7hn7k...,"{'also_bought': ['B0057794SA', 'B004LNSDB0', '...",NaN,"[[Home & Kitchen, Furniture, Kitchen & Dining ...",467.00,Coaster Home Furnishings


In [140]:
df_item_meta_data.iloc[ground[4]][['title', 'description']]

,title,description
767,Lipton Green Tea 100% Natural 100 Tea Bags Net...,"Pure and simple, our all-natural green tea is ..."


In [141]:
df_item_meta_data.iloc[rec_result[4]][['title', 'description']]

,title,description
6901,Harney &amp; Sons Peppermint Herbal Caffeine F...,Harney & Sons Peppermint Herbal Tea - 30 sache...
14768,"Emerald Harmony Original Trail Mix, 10-Ounce B...","Blend of peanuts, raisins, candy coated chocol..."
8436,"Keebler Cinnamon Roll, Original, 10-Ounce",Keebler Cinnamon Roll Cookies Original.New!Nat...
3817,Campbell's Chunky Grilled Steak Chili with Bea...,Campbell&#x2019;s&#xA0;Chunky is a great way t...
11705,Wine.com Milk Chocolate Gift Basket Containing...,Discover pure milk chocolate bliss for yoursel...
3119,Annie's Homegrown Shells &amp; White Cheddar M...,
17912,"Cheez-It Baked Snack Crackers, Whole Grain, 13...",
767,Lipton Green Tea 100% Natural 100 Tea Bags Net...,"Pure and simple, our all-natural green tea is ..."
927,"Twinings English Afternoon Tea, Tea Bags, 20-C...","Since 1706, Twinings has been at the forefront..."
4026,"Community Coffee Ground Coffee, Signature Dark...",
